In [27]:
import numpy as np
import datetime
import pytz
import pandas as pd
import sys
import os
import signals


In [3]:
minute = datetime.datetime(2024, 9, 20, 10, 0, 0, tzinfo=pytz.timezone('US/Eastern'))

In [13]:
a=minute.date()
a.year
b=datetime.date(a.year,a.month, a.day)
b

datetime.date(2024, 9, 20)

In [25]:
timestamp = pd.Timestamp('2024-01-02 09:31:00-0500', tz='UTC-05:00')
timestamp.normalize().tz_localize(None)
dt=datetime.date(2024,1,2)
dt==timestamp

False

In [26]:
 datetime.datetime(2024, 9, 20, 10, 0, 0, tzinfo=pytz.timezone('US/Eastern'))


datetime.datetime(2024, 9, 20, 10, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>)

In [28]:
class Recorder:
    def __init__(self):
        self.history = []

    def log(self, timestamp, position, pnl, risk):
        self.history.append({
            'timestamp': timestamp,
            'position': position,
            'pnl': pnl,
            'risk': risk
        })

    def reset(self):
        self.history = []  # Clears history between episodes

    def get_history(self):
        return pd.DataFrame(self.history)

In [ ]:
my_recorder = Recorder()
my_recorder.log(
    timestamp=minute,
    position=0,
    pnl=0,
    risk=0
)

In [33]:
import pandas as pd

portfolio = pd.DataFrame({
    'instrument': ['AAPL', 'SPX', 'MSFT'],
    'type': ['call', 'put', 'call'],
    'strike': [150, 4500, 320],
    'expiry': [pd.Timestamp('2024-04-19')] * 3,
    'quantity': [10, -5, 20]
})

In [ ]:
class PortfolioPricer:
    def __init__(self, pricing_function):
        self.pricing_function = pricing_function  # Your existing vectorized function

    def price_portfolio(self, portfolio_df, spot_price, vol, risk_free_rate, valuation_date):
        prices = self.pricing_function(
            spot=spot_price,
            types=portfolio_df['type'].values,
            strikes=portfolio_df['strike'].values,
            expiry=portfolio_df['expiry'].values,
            valuation_date=valuation_date,
            option_types=portfolio_df['type'].values,
            vol=vol,
            risk_free_rate=risk_free_rate
        )
        
        portfolio_df = portfolio_df.copy()
        portfolio_df['price'] = prices
        portfolio_df['value'] = portfolio_df['price'] * portfolio_df['quantity']
        return portfolio_df

    def total_portfolio_value(self, priced_portfolio_df):
        return priced_portfolio_df['value'].sum()

In [ ]:
class Portfolio:
    def __init__(self, portfolio_df, pricer):
        self.portfolio_df = portfolio_df
        self.pricer = pricer

    def price(self, spot_price, vol, risk_free_rate, valuation_date):
        priced_portfolio = self.pricer.price_portfolio(
            self.portfolio_df,
            spot_price,
            vol,
            risk_free_rate,
            valuation_date
        )
        return priced_portfolio

    def total_value(self, priced_portfolio_df):
        return self.pricer.total_portfolio_value(priced_portfolio_df)

In [38]:
# Example market parameters

import py_vollib_vectorized

def price_instrument(cp, strike, spot, texp, vol):
    #if self.debug:
    #    print(f"cp={cp}\n, strike={strike}\n, spot={spot}\n, texp={texp}\n, vol={vol}\n")
    #print(f"pricing_insturment sizes: cp={cp}, strike={strike.shape}, spot={spot.shape}, texp={texp.shape}, vol={vol.shape}")
    return py_vollib_vectorized.models.vectorized_black_scholes(cp, spot, strike, texp, 0, vol,return_as="numpy")


spot_price = 4400
vol = 0.20
risk_free_rate = 0.02
valuation_date = pd.Timestamp('2024-04-01')

# Instantiate pricer with your vectorized function
pricer = PortfolioPricer(pricing_function=price_instrument)


# Price portfolio
priced_portfolio = pricer.price_portfolio(portfolio, spot_price, vol, risk_free_rate, valuation_date)
total_value = pricer.total_portfolio_value(priced_portfolio)

print(priced_portfolio)
print(f'Total Portfolio Value: {total_value}')

TypeError: price_instrument() got an unexpected keyword argument 'strikes'

In [43]:

types=portfolio['type'].values
types=[x[0] for x in types]
print(types)

['c', 'p', 'c']


In [46]:
from dataclasses import dataclass, field
from typing import List, Dict
@dataclass
class OptionPosition:
    underlying: str
    option_type: str
    strike: float
    expiry: pd.Timestamp
    quantity: int
    price_history: pd.Series = field(default_factory=lambda: pd.Series(dtype='float64'))

    def record_price(self, timestamp, price):
        self.price_history.loc[timestamp] = price

@dataclass
class Portfolio:
    positions: List[OptionPosition] = field(default_factory=list)

    def add_position(self, position: OptionPosition):
        self.positions.append(position)

    def record_prices(self, timestamp, prices: Dict[str, float]):
        for pos in self.positions:
            key = f"{pos.underlying}_{pos.option_type}_{pos.strike}_{pos.expiry.date()}"
            if key in prices:
                pos.record_price(timestamp, prices[key])

# Usage example:
portfolio = Portfolio()
pos = OptionPosition('SPX', 'call', 4500, pd.Timestamp('2024-04-19'), 10)
portfolio.add_position(pos)

portfolio.record_prices(pd.Timestamp('2024-03-25 09:30'), {'SPX_call_4500_2024-04-19': 45.5})
portfolio.record_prices(pd.Timestamp('2024-03-25 09:31'), {'SPX_call_4500_2024-04-19': 46.0})

print(pos.price_history)

2024-03-25 09:30:00    45.5
2024-03-25 09:31:00    46.0
dtype: float64


In [70]:
import pandas as pd

class Portfolio:
    def __init__(self):
       self.positions = pd.DataFrame({
            'instrument_type': pd.Series(dtype='str'),
            'underlying': pd.Series(dtype='str'),
            'quantity': pd.Series(dtype='float'),
            'option_type': pd.Series(dtype='str'),
            'strike': pd.Series(dtype='float'),
            'expiry': pd.Series(dtype='datetime64[ns]')
})

    def add_position(self, instrument_type, underlying, quantity, option_type=pd.NA, strike=pd.NA, expiry=pd.NaT):
        expiry = pd.Timestamp(expiry) if expiry else None

        mask = (
            (self.positions['instrument_type'] == instrument_type) &
            (self.positions['underlying'] == underlying) &
            (self.positions['option_type'] == option_type) &
            (self.positions['strike'] == strike) &
            (self.positions['expiry'] == expiry)
        )

        if mask.any():
            self.positions.loc[mask, 'quantity'] += quantity
        else:
            if (instrument_type == 'option'):
                new_row = {
                    'instrument_type': instrument_type,
                    'underlying': underlying,
                    'quantity': quantity,
                    'option_type': option_type,
                    'strike': strike,
                    'expiry': expiry
                }
            elif (instrument_type == 'stock'):
                new_row = {
                    'instrument_type': instrument_type,
                    'underlying': underlying,
                    'quantity': quantity
                }
            #print(f"Adding new position: {new_row}")
            self.positions = pd.concat([self.positions, pd.DataFrame([new_row])], ignore_index=True)

    def add_option(self, underlying, quantity, option_type, strike, expiry):
        self.add_position('option', underlying, quantity, option_type, strike, expiry)

    def add_stock(self, underlying, quantity):
        self.add_position('stock', underlying, quantity)

    def get_positions(self):
        return self.positions

In [1]:
import pandas as pd
import datetime
import pytz

class TradeLedger:
    def __init__(self):
        self.trades_list = []

    def record_trade(self, timestamp, action, instrument_type, underlying, option_type, strike, expiry, quantity, price):
        new_trade = {
            'timestamp': timestamp,
            'action': action,  # 'buy' or 'sell'
            'instrument_type': instrument_type,
            'underlying': underlying,
            'option_type': option_type,
            'strike': strike,
            'expiry': expiry,
            'quantity': quantity,
            'price': price
        }
        self.trades_list.append(new_trade)

    @property
    def trades(self):
        #return pd.DataFrame(self.trades_list)
        return pd.DataFrame(self.trades_list).sort_values(by='timestamp').reset_index(drop=True)
        #return self.trades_list


class Portfolio:
    def __init__(self, ledger: TradeLedger):
        self.ledger = ledger
        self.positions = pd.DataFrame({
            'instrument_type': pd.Series(dtype='str'),
            'underlying': pd.Series(dtype='str'),
            'quantity': pd.Series(dtype='float'),
            'option_type': pd.Series(dtype='str'),
            'strike': pd.Series(dtype='float'),
            'expiry': pd.Series(dtype='datetime64[ns]')
        })

    def add_position(self, timestamp, instrument_type, underlying, quantity, price, option_type=None, strike=None, expiry=None):
        expiry = pd.Timestamp(expiry) if expiry else pd.NaT
        expiry = expiry.replace(hour=16, minute=17) if not pd.isna(expiry) else expiry

        mask = (
            (self.positions['instrument_type'] == instrument_type) &
            (self.positions['underlying'] == underlying) &
            (self.positions['option_type'] == option_type) &
            (self.positions['strike'] == strike) &
            (self.positions['expiry'] == expiry)
        )

        if mask.any():
            print(f"Updating existing position: {self.positions[mask]}")
            self.positions.loc[mask, 'quantity'] += quantity
        else:
            if (instrument_type == 'option'):
                new_row = {
                    'instrument_type': instrument_type,
                    'underlying': underlying,
                    'quantity': quantity,
                    'option_type': option_type,
                    'strike': strike,
                    'expiry': expiry
                }
            elif (instrument_type == 'stock'):
                new_row = {
                    'instrument_type': instrument_type,
                    'underlying': underlying,
                    'quantity': quantity
                }
            print(f"Adding new position: {new_row}")
            self.positions = pd.concat([self.positions, pd.DataFrame([new_row])])
            #self.positions = pd.concat([self.positions, pd.DataFrame([new_row])], ignore_index=True)

        if quantity > 0:
            action = 'buy'
        elif quantity < 0:
            action = 'sell'
        else:
            action = 'none'

        if action != 'none':
            self.ledger.record_trade(timestamp, action, instrument_type, underlying, option_type, strike, expiry, quantity, price)

    def add_option(self, timestamp, underlying, quantity, option_type, strike, expiry, price):
        self.add_position(timestamp, 'option', underlying, quantity, price, option_type, strike, expiry)

    def add_stock(self, timestamp, underlying, quantity, price):
        self.add_position(timestamp, 'stock', underlying, quantity, price)

    def get_positions(self):
        return self.positions

    def get_ledger(self):
        return self.ledger.trades

In [2]:
ledger= TradeLedger()
portfolio = Portfolio(ledger=ledger)

t = pd.Timestamp(datetime.datetime.now(tz=pytz.timezone('US/Eastern')))
# Initial positions
portfolio.add_option(t,'SPX', 10,'call', 4500, '2024-04-19',15)
portfolio.add_stock(t,'MSFT',15, 200)

# Add more positions (existing)
portfolio.add_option(t,'SPX', 5,'call', 4500, '2024-04-19',25)
portfolio.add_option(t,'SPX', 5,'call', 4400, '2024-04-19',25)
print("added the new instruments")
#portfolio.add_stock(t,'MSFT', -20,200)

# Check positions
#print(portfolio.get_positions())
print(portfolio.get_ledger())

Adding new position: {'instrument_type': 'option', 'underlying': 'SPX', 'quantity': 10, 'option_type': 'call', 'strike': 4500, 'expiry': Timestamp('2024-04-19 16:17:00')}
Adding new position: {'instrument_type': 'stock', 'underlying': 'MSFT', 'quantity': 15}
Updating existing position:   instrument_type underlying  quantity option_type  strike              expiry
0          option        SPX      10.0        call  4500.0 2024-04-19 16:17:00
Adding new position: {'instrument_type': 'option', 'underlying': 'SPX', 'quantity': 5, 'option_type': 'call', 'strike': 4400, 'expiry': Timestamp('2024-04-19 16:17:00')}
added the new instruments
                         timestamp action instrument_type underlying  \
0 2025-03-28 12:40:48.020717-04:00    buy          option        SPX   
1 2025-03-28 12:40:48.020717-04:00    buy           stock       MSFT   
2 2025-03-28 12:40:48.020717-04:00    buy          option        SPX   
3 2025-03-28 12:40:48.020717-04:00    buy          option        SPX   

In [3]:
import numpy as np
import signals
options_df=portfolio.get_positions()
options_df = options_df[options_df['instrument_type'] == 'option']
num_options=options_df.shape[0]  # Count the number of option positions
start_time = pd.Timestamp("2024-04-19").replace(hour=9, minute=31).tz_localize('US/Eastern')
end_time=pd.Timestamp("2024-04-19").replace(hour=15, minute=46).tz_localize("US/Eastern")
all_times = pd.date_range(start=start_time, end=end_time, freq="min")
num_times=len(all_times)
texp_vec=np.zeros(num_options*num_times)
i=0
all_texp=[]
price_results=[]
#for  (ind,row) in options_df.iterrows():
price_df = pd.DataFrame(index=all_times)
for row in options_df.itertuples():
    #texp_vec[i:i+num_times] =- (all_times - row.expiry.tz_localize("US/Eastern")).total_seconds().to_numpy() / (365 * 24 * 60 * 60)  # Convert to years
    #all_texp.append(texp_vec[i:i+num_times])
    all_texp=- (all_times - row.expiry.tz_localize("US/Eastern")).total_seconds().to_numpy() / (365 * 24 * 60 * 60) 

    instrument_label = f"{row.underlying}_{row.option_type}_{row.strike}_{row.expiry.date()}"
    #all_texp=texp_vec[i:i+num_times]
    all_strikes=np.full(num_times, row.strike)
    all_spots=np.full(num_times, 4400.0)
    all_vols=np.full(num_times, 0.2)
    all_types=np.full(num_times, row.option_type[0])
    all_prices=signals.price_instrument(all_types,  all_spots, all_strikes,all_texp , all_vols)
    price_results.append(all_prices)
    price_df[instrument_label]=all_prices

    i=i+1
#texp_vec2=np.concatenate(all_texp)
price_results2=np.concatenate(price_results)



In [6]:
price_df

,SPX_call_4500.0_2024-04-19,SPX_call_4400.0_2024-04-19
2024-04-19 09:31:00-04:00,100.000146,9.757252
2024-04-19 09:32:00-04:00,100.000143,9.745228
2024-04-19 09:33:00-04:00,100.000139,9.733190
2024-04-19 09:34:00-04:00,100.000136,9.721136
2024-04-19 09:35:00-04:00,100.000133,9.709068
...,...,...
2024-04-19 15:42:00-04:00,100.000000,2.864831
2024-04-19 15:43:00-04:00,100.000000,2.823608
2024-04-19 15:44:00-04:00,100.000000,2.781775
2024-04-19 15:45:00-04:00,100.000000,2.739303


In [35]:
a=- (all_times - row['expiry'].tz_localize("US/Eastern")).total_seconds().to_numpy() / (365 * 24 * 60 * 60) 

In [36]:
type(a)

numpy.ndarray

In [121]:
a=portfolio.get_positions()
a

,instrument_type,underlying,quantity,option_type,strike,expiry
0,option,SPX,15.0,call,4500.0,2024-04-19
0,option,SPX,5.0,call,4400.0,2024-04-19


In [5]:
import gymnasium as gym  # ✅ Use gymnasium instead of gym
import numpy as np
import pandas as pd
from gymnasium import spaces
from copy import deepcopy
from datetime import datetime, timedelta
import random
import pytz

import matplotlib.pyplot as plt
import matplotlib.animation as animation





class TemplateEnviroment(gym.Env):
    def __init__(self):
        super(TemplateEnviroment, self).__init__()
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 1), dtype=np.float32)
        self.episode_duration = timedelta(minutes=180)
        #self.current_time = self.pick_initial_datetime()
        #initial_state = self.data.loc[self.current_time]


        #define the initial state



    def reset(self):
        self.current_time = pick_random_datetime(datetime(2021, 1, 1), datetime(2021, 12, 31), timedelta(hours=9), timedelta(hours=16)-self.episode_duration)
        self.end_time = self.current_time + self.episode_duration
        state = self._get_state()
        info = self._get_info()
        return state, info


    def step(self, action):
        self.current_time += timedelta(minutes=30)
        done = self.current_time >= self.end_time
        reward = 0
        state = self._get_state()
        info = self._get_info()
        truncated = False
        return state, reward, done, truncated, info


    def _get_state(self):
        time_remaining= (self.end_time - self.current_time)/self.episode_duration
        return np.array([time_remaining])

    def _state_to_dict(self, state):
        return {"time_remaining": state[0]}

    def _get_info(self):
        return {}


    """

    def render(self, mode='human'):
        state_info = {
            "current_time": self.current_time.isoformat(),
            "end_time": self.end_time.isoformat()
        }
        print(state_info)

    """

    def render(self, mode='human'):
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.barh(['Time Remaining'], [self._get_state()[0]], color='blue')
        ax.set_xlim(0, 1)
        ax.set_xlabel('Normalized Time Remaining')
        ax.set_title(f'Time: {self.current_time.strftime("%H:%M")}')
        plt.tight_layout()

        if mode == 'human':
            plt.show()
            plt.close(fig)
        elif mode == 'rgb_array':
            fig.canvas.draw()
            width, height = fig.canvas.get_width_height()
            image = np.asarray(fig.canvas.renderer.buffer_rgba())
            plt.close(fig)
            return image[:, :, :3]
        plt.close(fig)

    def close(self):
        pass


    def rollout(self):
        state, _ = self.reset()
        done = False
        states, rewards, actions, infos = [], [], [], []
        while not done:
            action = self.action_space.sample()
            state1, reward, done, truncated, info = self.step(action)
            states.append(state)
            rewards.append(reward)
            actions.append(action)
            infos.append(info)
            state=state1

        states.append(state)
        return states, rewards, actions, infos, done, truncated

    def pick_initial_datetime(self):
        valid_start_times = self.data.index[self.data.index <= self.data.index.max() - self.episode_duration]
        initial_datetime = np.random.choice(valid_start_times)
        return initial_datetime

def pick_random_datetime(start_date: datetime, end_date: datetime, start_time: timedelta, end_time: timedelta,tz_str='America/New_York'):
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    while (not is_valid_date(random_date)):
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

    random_seconds = random.randint(int(start_time.total_seconds()), int(end_time.total_seconds()))//60*60
    random_datetime = datetime.combine(random_date, datetime.min.time()) + timedelta(seconds=random_seconds)
    random_datetime = pytz.timezone(tz_str).localize(random_datetime)
    return random_datetime




def is_valid_date(dt_datetime: datetime) -> bool:
    if dt_datetime.weekday()>4:
        return False
    return True

def run_episode(env):
    env.reset()
    done = False
    states=[]
    rewards=[]
    actions=[]
    truncates=[]
    infos=[]

    
    while not done:
        action = env.action_space.sample()
        state, reward, done, truncated, info = env.step(action)
        print(f"State: {state}, Reward: {reward}, Done: {done}, Info: {info}")
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        infos.append(info)
        if done:
            break
    return states, rewards, actions, infos



In [ ]:
import gymnasium as gym  # ✅ Use gymnasium instead of gym
import numpy as np
import pandas as pd
from gymnasium import spaces
from copy import deepcopy
from datetime import datetime, timedelta
import random
import pytz

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import signals



class StraddleEnvironment(gym.Env):
    def __init__(self):
        super(TemplateEnviroment, self).__init__()
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 1), dtype=np.float32)
        self.episode_duration = timedelta(minutes=180)
        data, underlying_data = signals.get_data()
        market_data = signals.get_market_data()
        #self.current_time = self.pick_initial_datetime()
        #initial_state = self.data.loc[self.current_time]


        #define the initial state



    def reset(self):
        self.current_time = pick_random_datetime(datetime(2021, 1, 1), datetime(2021, 12, 31), timedelta(hours=9), timedelta(hours=16)-self.episode_duration)
        self.end_time = self.current_time + self.episode_duration
        state = self._get_state()
        info = self._get_info()
        return state, info


    def step(self, action):
        self.current_time += timedelta(minutes=30)
        self.market_data.increment_minute(30)
        done = self.current_time >= self.end_time
        reward = 0
        state = self._get_state()
        info = self._get_info()
        truncated = False
        return state, reward, done, truncated, info


    def _get_state(self):
        time_remaining= (self.end_time - self.current_time)/self.episode_duration
        return np.array([time_remaining])

    def _state_to_dict(self, state):
        return {"time_remaining": state[0]}

    def _get_info(self):
        return {}


    """

    def render(self, mode='human'):
        state_info = {
            "current_time": self.current_time.isoformat(),
            "end_time": self.end_time.isoformat()
        }
        print(state_info)

    """

    def render(self, mode='human'):
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.barh(['Time Remaining'], [self._get_state()[0]], color='blue')
        ax.set_xlim(0, 1)
        ax.set_xlabel('Normalized Time Remaining')
        ax.set_title(f'Time: {self.current_time.strftime("%H:%M")}')
        plt.tight_layout()

        if mode == 'human':
            plt.show()
            plt.close(fig)
        elif mode == 'rgb_array':
            fig.canvas.draw()
            width, height = fig.canvas.get_width_height()
            image = np.asarray(fig.canvas.renderer.buffer_rgba())
            plt.close(fig)
            return image[:, :, :3]
        plt.close(fig)

    def close(self):
        pass


    def rollout(self):
        state, _ = self.reset()
        done = False
        states, rewards, actions, infos = [], [], [], []
        while not done:
            action = self.action_space.sample()
            state1, reward, done, truncated, info = self.step(action)
            states.append(state)
            rewards.append(reward)
            actions.append(action)
            infos.append(info)
            state=state1

        states.append(state)
        return states, rewards, actions, infos, done, truncated

    def pick_initial_datetime(self):
        valid_start_times = self.data.index[self.data.index <= self.data.index.max() - self.episode_duration]
        initial_datetime = np.random.choice(valid_start_times)
        return initial_datetime

def pick_random_datetime(start_date: datetime, end_date: datetime, start_time: timedelta, end_time: timedelta,tz_str='America/New_York'):
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    while (not is_valid_date(random_date)):
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

    random_seconds = random.randint(int(start_time.total_seconds()), int(end_time.total_seconds()))//60*60
    random_datetime = datetime.combine(random_date, datetime.min.time()) + timedelta(seconds=random_seconds)
    random_datetime = pytz.timezone(tz_str).localize(random_datetime)
    return random_datetime




def is_valid_date(dt_datetime: datetime) -> bool:
    if dt_datetime.weekday()>4:
        return False
    return True

def run_episode(env):
    env.reset()
    done = False
    states=[]
    rewards=[]
    actions=[]
    truncates=[]
    infos=[]

    
    while not done:
        action = env.action_space.sample()
        state, reward, done, truncated, info = env.step(action)
        print(f"State: {state}, Reward: {reward}, Done: {done}, Info: {info}")
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        infos.append(info)
        if done:
            break
    return states, rewards, actions, infos


In [1]:
from signals import ZeroDTESurfaceLoader, MarketData
import datetime
from zoneinfo import ZoneInfo
#import pytz
loader = ZeroDTESurfaceLoader("./algo_data/vol_surfaces2.csv","./algo_data/spy_daily_prices.csv")
data, underlying_data = loader.load_data()

print(f'main loaded data')
#print(data.head())
metadata = loader.get_metadata(data)
print(metadata)
market_data = MarketData(data, underlying_data)
#print(market_data.get_current_state("2023-09-20 10:00:00-04:00"))
#create a date time minute NY time zone
#minute = datetime.datetime(2024, 9, 20, 10,0 , 0, tzinfo=pytz.timezone('US/Eastern'))
#minute=pytz.timezone('US/Eastern').localize(datetime.datetime(2024, 9, 20, 10, 0, 0))
minute=datetime.datetime(2024, 9, 20, 10, 0, 0, tzinfo=ZoneInfo("US/Eastern"))
#print(f"current minute: {minute}")
#print(market_data.get_current_row(minute))
market_data.set_current_minute(minute)
#market_data.set_current_minute(minute)

row=market_data.get_current_row()

data validated


/Users/stirlitz/ncGitHub/daily_options/daily_options1/signals.py:208: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['minute'] = pd.to_datetime(df['minute'],errors="coerce")


0         2024-01-02
1         2024-01-02
2         2024-01-02
3         2024-01-02
4         2024-01-02
             ...    
112994    2025-03-03
112995    2025-03-03
112996    2025-03-03
112997    2025-03-03
112998    2025-03-03
Name: date, Length: 112999, dtype: object
main loaded data
{'start_date': Timestamp('2024-01-02 09:31:00-0500', tz='UTC-05:00'), 'end_date': Timestamp('2025-03-03 16:01:00-0500', tz='UTC-05:00'), 'num_records': 112999}


In [2]:
import pandas as pd
temp_minute = pd.Timestamp(minute)
#market_data.all_dfs
temp_minute=temp_minute.normalize()
market_data.all_dfs[temp_minute]

,minute,implied_spot,atm_vol,slope,quadratic_term,scaled_slope,scaled_quadratic,open_price,high_price,low_price,...,max_atm_vol_60,mean_atm_vol_60,mean_price_60,mean_price_10,under_close_shifted,under_open,under_hvol_20,under_min_price_20,under_max_price_20,under_mean_price_20
70380,2024-09-20 09:31:00-04:00,567.844972,0.266482,-3.972185,165.508420,-0.011036,0.127760,567.8400,568.2300,567.810,...,0.314945,0.260488,571.226560,570.473193,570.98,567.84,0.146151,540.36,570.98,557.7535
70381,2024-09-20 09:32:00-04:00,567.841998,0.265169,-4.022472,170.182399,-0.011162,0.131044,567.8300,567.8700,567.620,...,0.314945,0.260797,571.163498,570.209364,570.98,567.84,0.146151,540.36,570.98,557.7535
70382,2024-09-20 09:33:00-04:00,567.739694,0.267222,-3.985849,166.518473,-0.011047,0.127906,567.8800,568.0300,567.660,...,0.314945,0.261283,571.094256,569.931100,570.98,567.84,0.146151,540.36,570.98,557.7535
70383,2024-09-20 09:34:00-04:00,567.900078,0.269463,-4.171410,157.529688,-0.011547,0.120702,567.7400,567.9348,567.600,...,0.314945,0.261897,571.025258,569.624356,570.98,567.84,0.146151,540.36,570.98,557.7535
70384,2024-09-20 09:35:00-04:00,567.975055,0.263243,-4.092411,165.712486,-0.011314,0.126657,567.8800,568.1695,567.815,...,0.314945,0.262306,570.960534,569.335328,570.98,567.84,0.146151,540.36,570.98,557.7535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70766,2024-09-20 15:57:00-04:00,567.905563,0.263794,-13.410523,2944.093051,-0.008270,0.111951,568.0300,568.1300,567.800,...,0.304494,0.239186,568.062487,568.329181,570.98,567.84,0.146151,540.36,570.98,557.7535
70767,2024-09-20 15:58:00-04:00,567.997138,0.246029,-5.011367,6468.254047,-0.003012,0.233662,567.9000,568.1200,567.860,...,0.304494,0.239775,568.070566,568.279804,570.98,567.84,0.146151,540.36,570.98,557.7535
70768,2024-09-20 15:59:00-04:00,568.092202,0.237572,-3.223026,5311.500137,-0.001885,0.181776,567.9599,568.1200,567.830,...,0.304494,0.240215,568.078546,568.228970,570.98,567.84,0.146151,540.36,570.98,557.7535
70769,2024-09-20 16:00:00-04:00,568.034805,0.214337,-10.570512,6854.366558,-0.006010,0.221546,568.0550,568.2200,567.800,...,0.304494,0.240299,568.087399,568.181121,570.98,567.84,0.146151,540.36,570.98,557.7535


In [ ]:
row.keys()

Index(['minute', 'implied_spot', 'atm_vol', 'slope', 'quadratic_term',
       'scaled_slope', 'scaled_quadratic', 'open_price', 'high_price',
       'low_price', 'close_price', 'volume', 'vwap', 'timestamp',
       'transactions', 'otc', 'timestamp_utc', 'timestamp_est', 'date',
       'years_to_maturity', 'straddle_price', 'pct_straddle_price',
       'start_of_day', 'return', 'hvol_60', 'min_price_60', 'max_price_60',
       'min_atm_vol_60', 'max_atm_vol_60', 'mean_atm_vol_60', 'mean_price_60',
       'mean_price_10', 'under_close_shifted', 'under_open', 'under_hvol_20',
       'under_min_price_20', 'under_max_price_20', 'under_mean_price_20'],
      dtype='object')

In [3]:
market_data.get_current_row()
market_data.increment_minute(2)
market_data.get_current_row()
market_data.missing_minutes

{}

In [8]:
import pandas as pd
df=pd.read_csv("./algo_data/vol_surfaces2.csv")


In [9]:

df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.tz_localize(ZoneInfo("US/Eastern"))
df['date']

0        2024-01-02 00:00:00-05:00
1        2024-01-02 00:00:00-05:00
2        2024-01-02 00:00:00-05:00
3        2024-01-02 00:00:00-05:00
4        2024-01-02 00:00:00-05:00
                    ...           
112994   2025-03-03 00:00:00-05:00
112995   2025-03-03 00:00:00-05:00
112996   2025-03-03 00:00:00-05:00
112997   2025-03-03 00:00:00-05:00
112998   2025-03-03 00:00:00-05:00
Name: date, Length: 112999, dtype: datetime64[ns, US/Eastern]

In [8]:
import pandas as pd
temp_date = pd.Timestamp(minute.date())
dt=datetime.date(temp_date.year, temp_date.month, temp_date.day)
market_data.df["minute"]==minute


0         False
1         False
2         False
3         False
4         False
          ...  
112994    False
112995    False
112996    False
112997    False
112998    False
Name: minute, Length: 112999, dtype: bool

In [7]:
data = pd.read_csv("./algo_data/vol_surfaces2.csv") 
data["date"] =pd.to_datetime(data["date"], utc=False)
data

,minute,implied_spot,atm_vol,slope,quadratic_term,scaled_slope,scaled_quadratic,open_price,high_price,low_price,close_price,volume,vwap,timestamp,transactions,otc,timestamp_utc,timestamp_est,date
0,2024-01-02 09:31:00-05:00,472.563933,0.189797,-0.386711,133.447533,-0.001074,0.103011,472.160,472.67,472.050,472.5300,1190522.0,472.9271,1704205800000,11700,NaN,2024-01-02 14:30:00+00:00,2024-01-02 09:30:00-05:00,2024-01-02
1,2024-01-02 09:32:00-05:00,472.705855,0.183684,-0.393608,156.933023,-0.001092,0.120842,472.530,472.70,472.365,472.6812,756655.0,473.3993,1704205860000,3810,NaN,2024-01-02 14:31:00+00:00,2024-01-02 09:31:00-05:00,2024-01-02
2,2024-01-02 09:33:00-05:00,472.784149,0.181348,-0.452450,182.755226,-0.001254,0.140378,472.680,472.80,472.650,472.7800,693889.0,473.6179,1704205920000,3807,NaN,2024-01-02 14:32:00+00:00,2024-01-02 09:32:00-05:00,2024-01-02
3,2024-01-02 09:34:00-05:00,472.743159,0.180591,-0.333196,179.169754,-0.000922,0.137283,472.760,472.77,472.480,472.6900,404264.0,472.6133,1704205980000,3280,NaN,2024-01-02 14:33:00+00:00,2024-01-02 09:33:00-05:00,2024-01-02
4,2024-01-02 09:35:00-05:00,472.689797,0.178740,-0.421782,178.336217,-0.001166,0.136305,472.685,472.78,472.550,472.6700,250698.0,472.6734,1704206040000,2284,NaN,2024-01-02 14:34:00+00:00,2024-01-02 09:34:00-05:00,2024-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112994,2025-03-03 15:57:00-05:00,583.217755,0.433006,-0.643380,2250.084372,-0.000397,0.085561,583.120,583.38,583.080,583.1900,531037.0,583.2175,1741035360000,5290,NaN,2025-03-03 20:56:00+00:00,2025-03-03 15:56:00-05:00,2025-03-03
112995,2025-03-03 15:58:00-05:00,583.318760,0.399562,-1.119237,2824.657505,-0.000673,0.102039,583.220,583.42,582.800,583.3300,694988.0,583.1694,1741035420000,6733,NaN,2025-03-03 20:57:00+00:00,2025-03-03 15:57:00-05:00,2025-03-03
112996,2025-03-03 15:59:00-05:00,583.589449,0.375949,-6.439821,3647.324778,-0.003767,0.124823,583.320,583.61,583.115,583.5700,829842.0,584.9927,1741035480000,6584,NaN,2025-03-03 20:58:00+00:00,2025-03-03 15:58:00-05:00,2025-03-03
112997,2025-03-03 16:00:00-05:00,584.010249,0.336107,0.051465,3648.685396,0.000029,0.117932,583.570,584.11,583.200,583.9700,2382661.0,585.3033,1741035540000,12913,NaN,2025-03-03 20:59:00+00:00,2025-03-03 15:59:00-05:00,2025-03-03


In [3]:

import gymnasium as gym  # ✅ Use gymnasium instead of gym
import numpy as np
import pandas as pd
from gymnasium import spaces
from copy import deepcopy
from datetime import datetime, timedelta
import random
#import pytz

import matplotlib.pyplot as plt
import matplotlib.animation as animation





class StraddleEnvironment(gym.Env):
    def __init__(self):
        super(StraddleEnvironment, self).__init__()
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 1), dtype=np.float32)
        self.episode_duration = timedelta(minutes=180)
        #self.current_time = self.pick_initial_datetime()
        #initial_state = self.data.loc[self.current_time]
        loader = ZeroDTESurfaceLoader("./algo_data/vol_surfaces2.csv","./algo_data/spy_daily_prices.csv")
        data, underlying_data = loader.load_data()
        market_data= MarketData(data, underlying_data)
        self.market_data=market_data
        self.metadata = loader.get_metadata(data)
        self.start_date = self.metadata["start_date"]
        self.end_date = self.metadata["end_date"]

        


        #define the initial state



    def reset(self):
        self.current_time = pick_random_datetime(self.start_date,self.end_date, timedelta(hours=9,minutes=31), timedelta(hours=16,minutes=1)-self.episode_duration,self.market_data)
        #print(f"current time: {self.current_time}")
        self.end_time = self.current_time + self.episode_duration
        state = self._get_state()
        info = self._get_info()
        self.market_data.set_current_minute(self.current_time)
        #print(f"current time: {self.current_time}")
        state= self.market_data.get_current_row()
        return state, info


    def step(self, action):
        self.current_time += timedelta(minutes=30)
        done = self.current_time >= self.end_time
        reward = 0
        state = self._get_state()
        info = self._get_info()
        truncated = False
        return state, reward, done, truncated, info


    def _get_state(self):
        time_remaining= (self.end_time - self.current_time)/self.episode_duration
        return np.array([time_remaining])

    def _state_to_dict(self, state):
        return {"time_remaining": state[0]}

    def _get_info(self):
        return {}


    """

    def render(self, mode='human'):
        state_info = {
            "current_time": self.current_time.isoformat(),
            "end_time": self.end_time.isoformat()
        }
        print(state_info)

    """

    def render(self, mode='human'):
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.barh(['Time Remaining'], [self._get_state()[0]], color='blue')
        ax.set_xlim(0, 1)
        ax.set_xlabel('Normalized Time Remaining')
        ax.set_title(f'Time: {self.current_time.strftime("%H:%M")}')
        plt.tight_layout()

        if mode == 'human':
            plt.show()
            plt.close(fig)
        elif mode == 'rgb_array':
            fig.canvas.draw()
            width, height = fig.canvas.get_width_height()
            image = np.asarray(fig.canvas.renderer.buffer_rgba())
            plt.close(fig)
            return image[:, :, :3]
        plt.close(fig)

    def close(self):
        pass


    def rollout(self):
        state, _ = self.reset()
        done = False
        states, rewards, actions, infos = [], [], [], []
        while not done:
            action = self.action_space.sample()
            state1, reward, done, truncated, info = self.step(action)
            states.append(state)
            rewards.append(reward)
            actions.append(action)
            infos.append(info)
            state=state1

        states.append(state)
        return states, rewards, actions, infos, done, truncated

    def pick_initial_datetime(self):
        valid_start_times = self.data.index[self.data.index <= self.data.index.max() - self.episode_duration]
        initial_datetime = np.random.choice(valid_start_times)
        return initial_datetime

def pick_random_datetime(start_date: datetime, end_date: datetime, start_time: timedelta, end_time: timedelta,market_data,tz_str='America/New_York'):
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    while (not is_valid_date(random_date) or (pd.Timestamp(random_date.date()).tz_localize(ZoneInfo("US/Eastern"))in market_data.missing_dates)):
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        #print(f"random date: {random_date}")
        #print(f"missing dates: {market_data.missing_dates}")
    

    random_seconds = random.randint(int(start_time.total_seconds()), int(end_time.total_seconds()))//60*60
    random_datetime = datetime.combine(random_date, datetime.min.time()) + timedelta(seconds=random_seconds)
    random_datetime = random_datetime.replace(tzinfo=ZoneInfo("US/Eastern"))
    #random_datetime = pytz.timezone(tz_str).localize(random_datetime)
    return random_datetime

"""
def pick_random_datetime(start_date: datetime, end_date: datetime, start_time: timedelta, end_time: timedelta,tz_str='America/New_York'):
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    while (not is_valid_date(random_date)):
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

    random_seconds = random.randint(int(start_time.total_seconds()), int(end_time.total_seconds()))//60*60
    random_datetime = datetime.combine(random_date, datetime.min.time()) + timedelta(seconds=random_seconds)
    random_datetime = random_datetime.replace(tzinfo=ZoneInfo("US/Eastern"))
    #random_datetime = pytz.timezone(tz_str).localize(random_datetime)
    return random_datetime
"""


def is_valid_date(dt_datetime: datetime) -> bool:
    if dt_datetime.weekday()>4:
        return False
    return True

def run_episode(env):
    env.reset()
    done = False
    states=[]
    rewards=[]
    actions=[]
    truncates=[]
    infos=[]

    
    while not done:
        action = env.action_space.sample()
        state, reward, done, truncated, info = env.step(action)
        print(f"State: {state}, Reward: {reward}, Done: {done}, Info: {info}")
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        infos.append(info)
        if done:
            break
    return states, rewards, actions, infos


In [4]:
env=StraddleEnvironment()

data validated


/Users/stirlitz/ncGitHub/daily_options/daily_options1/signals.py:208: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['minute'] = pd.to_datetime(df['minute'],errors="coerce")


0         2024-01-02
1         2024-01-02
2         2024-01-02
3         2024-01-02
4         2024-01-02
             ...    
112994    2025-03-03
112995    2025-03-03
112996    2025-03-03
112997    2025-03-03
112998    2025-03-03
Name: date, Length: 112999, dtype: object


In [53]:
bla=env.rollout()
bla[0][0]

minute                 2025-01-22 09:49:00-05:00
implied_spot                          605.870782
atm_vol                                 0.160146
slope                                  -5.228232
quadratic_term                        279.279321
scaled_slope                             -0.0142
scaled_quadratic                        0.206024
open_price                                605.67
high_price                               605.935
low_price                                 605.63
close_price                              605.919
volume                                   66011.0
vwap                                    605.8104
timestamp                          1737557280000
transactions                                1029
otc                                          NaN
timestamp_utc          2025-01-22 14:48:00+00:00
timestamp_est          2025-01-22 09:48:00-05:00
date                   2025-01-22 00:00:00-05:00
years_to_maturity                       0.000738
straddle_price      

In [28]:
action= random.sample(range(0,3), 1)[0]
action

1

In [ ]:
random_date = env.start_date + timedelta(days=random.randint(0, (env.end_date - env.start_date).days))


AttributeError: 'Timestamp' object has no attribute 'ZoneInfo'

In [16]:
env.reset()

current time: 2025-01-08 12:08:00-05:00


(minute                 2025-01-08 12:08:00-05:00
 implied_spot                          588.071568
 atm_vol                                 0.277605
 slope                                  -3.351328
 quadratic_term                        234.406529
 scaled_slope                           -0.007292
 scaled_quadratic                        0.110973
 open_price                                 588.3
 high_price                                588.34
 low_price                                 587.96
 close_price                               588.07
 volume                                   58899.0
 vwap                                    588.0929
 timestamp                          1736356020000
 transactions                                 949
 otc                                          NaN
 timestamp_utc          2025-01-08 17:07:00+00:00
 timestamp_est          2025-01-08 12:07:00-05:00
 date                   2025-01-08 00:00:00-05:00
 years_to_maturity                       0.000473


In [6]:
for i in range(10000):
    #print(f"Episode {i+1}")
    obs,_= env.reset()
print(obs)


minute                 2025-01-03 10:49:00-05:00
implied_spot                          588.919143
atm_vol                                 0.233765
slope                                  -4.016743
quadratic_term                        206.162899
scaled_slope                           -0.010031
scaled_quadratic                        0.128568
open_price                                 588.8
high_price                                589.03
low_price                                 588.67
close_price                               588.92
volume                                   35818.0
vwap                                    588.8645
timestamp                          1735919280000
transactions                                 745
otc                                          NaN
timestamp_utc          2025-01-03 15:48:00+00:00
timestamp_est          2025-01-03 10:48:00-05:00
date                   2025-01-03 00:00:00-05:00
years_to_maturity                       0.000624
straddle_price      

In [23]:
env.current_time
#env.market_data.missing_dates

datetime.datetime(2024, 3, 29, 12, 36, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern'))

In [35]:
env.current_time.date() in env.market_data.missing_dates

pd.Timestamp(env.current_time.date()).tz_localize(ZoneInfo("US/Eastern"))in env.market_data.missing_dates

True

In [81]:
env.current_time
any(env.market_data.df["date"]==env.current_date)
env.current_time.date() #in env.market_data.missing_dates

AttributeError: 'StraddleEnvironment' object has no attribute 'current_date'

In [24]:
env.market_data.missing_dates

[Timestamp('2024-01-15 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2024-02-19 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2024-03-29 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-05-27 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-06-19 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-07-03 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-07-04 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-09-02 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2024-11-28 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2024-11-29 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2024-12-24 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2024-12-25 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2025-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2025-01-09 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2025-01-20 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2025-02-17 00:00:00-0500', tz='US/Eastern')]

In [6]:
all_dates=env.market_data.df["date"].unique()
all_dates
#find missing weekdays
all_dates = pd.to_datetime(all_dates)
all_dates = sorted(all_dates)
date_range= pd.date_range(start=all_dates[0], end=all_dates[-1], freq='B')
missing_dates = []
for dt in date_range: 
    if dt not in all_dates:
        missing_dates.append(dt)
missing_dates

[Timestamp('2024-01-15 00:00:00'),
 Timestamp('2024-02-19 00:00:00'),
 Timestamp('2024-03-29 00:00:00'),
 Timestamp('2024-05-27 00:00:00'),
 Timestamp('2024-06-19 00:00:00'),
 Timestamp('2024-07-03 00:00:00'),
 Timestamp('2024-07-04 00:00:00'),
 Timestamp('2024-09-02 00:00:00'),
 Timestamp('2024-11-28 00:00:00'),
 Timestamp('2024-11-29 00:00:00'),
 Timestamp('2024-12-24 00:00:00'),
 Timestamp('2024-12-25 00:00:00'),
 Timestamp('2025-01-01 00:00:00'),
 Timestamp('2025-01-09 00:00:00'),
 Timestamp('2025-01-20 00:00:00'),
 Timestamp('2025-02-17 00:00:00')]

In [24]:

missing_minutes={}
for dt in date_range:
    if dt not in missing_dates:
        daily_df= env.market_data.df[env.market_data.df["date"]==dt]
        full_day_minutes = pd.date_range(
            start=dt.replace(hour=9, minute=31, tzinfo=ZoneInfo("US/Eastern")), 
            end=dt.replace(hour=16, minute=1, tzinfo=ZoneInfo("US/Eastern")),
            freq='min'
        )
        missing = full_day_minutes.difference(daily_df["minute"])
        if (len(missing)>0):
            missing_minutes[dt] = missing
#print(full_day_minutes)
print(missing_minutes)

{}


In [ ]:
all_dfs={}
for dt in missing_dates:
    daily_df= env.market_data.df[env.market_data.df["date"]==dt]
    all_dfs[dt]=daily_df
